In [22]:
import os
from pathlib import Path

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')
path = Path('data/archive/bodyfat_dataset.csv')


In [23]:
import torch, numpy as np, pandas as pd

df = pd.read_csv(path)

df

,title,url,meanPrediction,medianPrediction,bfPredictions,image_1,image_2,image_3,image_4,image_5
0,Leanest ive ever been. Never seen veins like this before.,https://www.reddit.com/gallery/1b6k5jh,8.00,8.0,[8],https://preview.redd.it/leanest-ive-ever-been-never-seen-veins-like-this-before-v0-hibkcr8hfdmc1.jpg?width=640&crop=smart&auto=webp&s=4604d036777deffcf28c2e2f396b19f77ac54c30,https://preview.redd.it/leanest-ive-ever-been-never-seen-veins-like-this-before-v0-cvvyjr8hfdmc1.jpg?width=640&crop=smart&auto=webp&s=bbc6e7207b1aae8893ecb76d1a68adc87fcd7aa2,https://preview.redd.it/leanest-ive-ever-been-never-seen-veins-like-this-before-v0-kev0ir8hfdmc1.jpg?width=640&crop=smart&auto=webp&s=3cf034949f12a0b62655c3dbcd0d77d4418c64fe,https://preview.redd.it/leanest-ive-ever-been-never-seen-veins-like-this-before-v0-yur9ot8hfdmc1.jpg?width=476&format=pjpg&auto=webp&s=5af3f2a2258f511a1606651727635f31f55f4d82,NaN
1,Let me know. 78kg :),https://i.redd.it/4occeq9wdd2c1.jpg,9.80,10.0,"[9, 10, 10, 9, 11]",https://i.redd.it/4occeq9wdd2c1.jpg,NaN,NaN,NaN,NaN
2,What is my bf% I believe it’s around 13-14%,https://i.redd.it/lvfmowq0zhh91.jpg,14.33,13.5,"[13, 19, 14, 14, 13, 13]",https://i.redd.it/lvfmowq0zhh91.jpg,NaN,NaN,NaN,NaN
3,"25F | 4'11"" | 107 lbs",https://www.reddit.com/gallery/1e30z2f,23.33,24.0,"[20, 26, 24]",https://preview.redd.it/25f-411-107-lbs-v0-4lmyvzck8hcd1.jpg?width=640&crop=smart&auto=webp&s=8ad8d28fdda2a0c57664d858e31e13889dfb008d,https://preview.redd.it/25f-411-107-lbs-v0-huczvrhk8hcd1.jpg?width=640&crop=smart&auto=webp&s=2445bde27112796d8e09d8825a1cb03d7bb83108,NaN,NaN,NaN
4,Bodyfat?,https://www.reddit.com/gallery/1ktwao9,17.80,18.0,"[18, 16, 18, 15, 22]",https://preview.redd.it/8ekhv0d4xl2f1.jpg?width=640&crop=smart&auto=webp&s=dfaa3a2d96c714e36a6b2579ad6559b098e145c7,https://preview.redd.it/rwkt50d4xl2f1.jpg?width=640&crop=smart&auto=webp&s=9d4a08637bbd189e9f44f47dfe20948be74f85d4,https://preview.redd.it/vau412d4xl2f1.jpg?width=640&crop=smart&auto=webp&s=610a51fef6107a8c7635afdc1722d01e620bf86b,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
801,[GMBF] 6'1 220LBS Any estimates appreciated!,https://i.redd.it/z543jfquva731.jpg,12.00,12.0,[12],https://i.redd.it/z543jfquva731.jpg,NaN,NaN,NaN,NaN
802,"[GMBF] (M/22/6'2""/195lbs)",https://i.imgur.com/PY44dK5.jpg,10.50,10.5,"[11, 9, 10, 12]",https://i.imgur.com/PY44dK5.jpg,NaN,NaN,NaN,NaN
803,[GMBF] M/26/6'1/191lbs - down from 245lbs,https://i.imgur.com/pWGT7nV.jpg,15.00,15.0,[15],https://i.imgur.com/pWGT7nV.jpg,NaN,NaN,NaN,NaN
804,[GMBF] (M/26/5’10”/153lbs to 168lbs) What would you estimate my bf% to be before and after 4 months of (hopefully lean) bulking?,https://i.redd.it/lc2q2svumrt11.jpg,12.00,12.0,"[10, 14]",https://i.redd.it/lc2q2svumrt11.jpg,NaN,NaN,NaN,NaN


First we must downlaod all the images from our dataframe using simple http fetching. The images are stores as row_image_number.filename

In [24]:
import requests
from tqdm import tqdm

def download_all_images(df, output_dir="images"):
    os.makedirs(output_dir, exist_ok=True)
    image_cols = [f"image_{i}" for i in range(1, 6)]
    
    seen_urls = set()
    image_count = 0

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        for col in image_cols:
            url = row.get(col)
            if isinstance(url, str) and url.startswith("http") and url not in seen_urls:
                try:
                    response = requests.get(url, timeout=10)
                    if response.status_code == 200:
                        file_ext = url.split('.')[-1].split('?')[0]
                        file_name = f"{idx}_{col}.{file_ext}"
                        file_path = os.path.join(output_dir, file_name)
                        with open(file_path, "wb") as f:
                            f.write(response.content)
                        seen_urls.add(url)
                        image_count += 1
                except Exception as e:
                    print(f"Error downloading {url}: {e}")

    print(f"\nDownloaded {image_count} unique images to '{output_dir}/'")

This maps our original dataset, where each row could have up to 5 images, into a dataset where each image has it's own row - and a coresponding body fat %. This prepares the data to be put through our model! 

In [25]:
def create_regression_csv(df, output_csv="image_labels.csv", label_col="meanPrediction", image_prefix="image_", output_dir="images"):
    # Ensure column names are stripped of whitespace
    df.columns = df.columns.str.strip()
    
    image_cols = [col for col in df.columns if col.startswith(image_prefix)]
    records = []

    for idx, row in df.iterrows():
        label = row[label_col]
        for col in image_cols:
            url = row.get(col)
            if isinstance(url, str) and url.startswith("http"):
                ext = url.split('.')[-1].split('?')[0].lower()
                ext = ext if ext in ['jpg', 'jpeg', 'png', 'webp'] else 'jpg'
                filename = f"{idx}_{col}.{ext}"
                records.append({"filename": filename, "target": label})
    
    df_out = pd.DataFrame(records)
    df_out.to_csv(output_csv, index=False)
    print(f"Created {output_csv} with {len(df_out)} labeled images")
    return df_out

Download and map!

In [26]:
download_all_images(df)  

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 806/806 [06:16<00:00,  2.14it/s]


Downloaded 1578 unique images to 'images/'


In [29]:
df_labels = create_regression_csv(df) 

Created image_labels.csv with 1597 labeled images


As we can see, our images have downloaded and are stored in a much more simple dataframe!

In [30]:
df_labels.head()

,filename,target
0,0_image_1.jpg,8.0
1,0_image_2.jpg,8.0
2,0_image_3.jpg,8.0
3,0_image_4.jpg,8.0
4,1_image_1.jpg,9.8


In [31]:
from fastai.vision.all import *

failed = verify_images(get_image_files(Path('cropped_images')))
failed.map(Path.unlink)
len(failed)

0

Split training and validation sets

In [33]:
from sklearn.model_selection import train_test_split

# Drop NaNs and build the initial label dictionary
initial_nan_labels_count = df_labels['target'].isna().sum()
if initial_nan_labels_count > 0:
    print(f"DEBUG: Found {initial_nan_labels_count} NaN values in 'target' column. Dropping rows with NaN targets.")
    df_labels.dropna(subset=['target'], inplace=True)
else:
    print("DEBUG: No NaN values found in 'target' column (good).")

# Split into train and validation
train_df, valid_df = train_test_split(df_labels, test_size=0.2, random_state=42)

train_label_dict = dict(zip(train_df['filename'], train_df['target']))
valid_label_dict = dict(zip(valid_df['filename'], valid_df['target']))
all_label_dict = {**train_label_dict, **valid_label_dict}

DEBUG: No NaN values found in 'target' column (good).


Now lets make the datablock. For augmentations, we'll do all except warp (that might make the phyisquese look too different). We can see some of our datablock's examples with show_batch.

In [38]:
print(f"DEBUG: Train labels: {len(train_label_dict)} | Valid labels: {len(valid_label_dict)}")

# Get all image files
path = Path('cropped_images')
print(f"DEBUG: Image path set to: {path}")

all_image_files = get_image_files(path)
print(f"DEBUG: Total image files found by get_image_files: {len(all_image_files)}")

# Filter image files to only those with matching labels
processable_image_files = [f for f in all_image_files if f.name in all_label_dict]
print(f"DEBUG: Processable image files (with matching labels): {len(processable_image_files)}")

# Safety check
if len(processable_image_files) == 0:
    print("CERROR: No processable image files found (no images match labels or vice-versa).")
    if all_image_files and all_label_dict:
        print(f"  Sample image file: {all_image_files[0].name}")
        print(f"  Sample label key: {next(iter(all_label_dict.keys()))}")
        if all_image_files[0].name not in all_label_dict and all_image_files[0].name.split('.')[0] in [k.split('.')[0] for k in all_label_dict.keys()]:
            print(" Filename extensions might differ between image files and label keys.")
    raise ValueError("Cannot create DataLoaders: No matching image files and labels.")

# Helper function to get label
def get_y_func(fn):
    key = fn.name
    if key not in all_label_dict:
        print(f"DEBUG ERROR: Label not found for: {key} during get_y_func call. This should not happen if pre-filtered.")
        raise ValueError(f"Label not found for: {key}")
    return all_label_dict[key]

# Generate index lists for DataBlock IndexSplitter
filename_to_index = {f.name: i for i, f in enumerate(processable_image_files)}
valid_idxs = [filename_to_index[fname] for fname in valid_df['filename'] if fname in filename_to_index]
splitter = IndexSplitter(valid_idxs)

def convert_to_rgb(img):
    return img.convert('RGB')

# Transformations
item_tfms = Resize(244, method="pad")

batch_tfms = aug_transforms(
    do_flip=True,
    max_rotate=2,     
    max_zoom=1.05,    
    max_lighting=0.1, 
    max_warp=0.,
    p_affine=0.3,     
    p_lighting=0.3   
)
dblock = DataBlock(
    blocks=(ImageBlock, RegressionBlock),
    get_items=lambda _: processable_image_files,
    splitter=splitter,
    get_y=get_y_func,
    item_tfms=item_tfms,
    batch_tfms=batch_tfms,
    n_inp=1
)

print("DEBUG: Attempting to create DataLoaders...")
try:
    dls = dblock.dataloaders(path, bs=16)
    print("DEBUG: DataLoaders created successfully.")
    print(f"DEBUG: Number of training batches: {len(dls.train)}")
    print(f"DEBUG: Number of validation batches: {len(dls.valid)}")
except Exception as e:
    print(f"CRITICAL ERROR: Failed to create DataLoaders: {e}")
    raise


DEBUG: Train labels: 1277 | Valid labels: 320
DEBUG: Image path set to: cropped_images
DEBUG: Total image files found by get_image_files: 1357
DEBUG: Processable image files (with matching labels): 1357
DEBUG: Attempting to create DataLoaders...
DEBUG: DataLoaders created successfully.
DEBUG: Number of training batches: 68
DEBUG: Number of validation batches: 17


Train it!

In [42]:
import timm
from fastai.vision.all import *

# Create model with correct input size
model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=1)

# Create learner with the pre-configured model
learn = Learner(dls, model, metrics=rmse)

In [43]:
learn.fine_tune(20)

epoch,train_loss,valid_loss,_rmse,time
0,42.501873,143.486038,11.978566,00:56


epoch,train_loss,valid_loss,_rmse,time
0,12.112394,12.197266,3.492458,00:51
1,7.454126,10.838336,3.292163,00:58
2,5.753793,14.980948,3.870523,00:58
3,7.135998,31.817371,5.640689,00:57
4,7.226499,3602.582764,60.021519,00:53
5,7.059509,18.491421,4.300165,00:53
6,6.461150,11.648339,3.412966,00:56
7,6.213525,13.621491,3.690731,00:58
8,4.447729,8.372024,2.893445,00:57
9,3.955762,18.085747,4.252734,00:58


Now we have a working model! For example, it predicts this picture at 13% bodyfat (not so far off in my opinion)

In [13]:
bf,_,probs = learn.predict(PILImage.create('images/248_image_2.jpg'))
print(f"Bodyfat prediction: {probs[0]:.4f}")

Bodyfat prediction: 14.5279


Finally, export the model

In [15]:
learn.export('model.pkl')

In [21]:
import sys
print(sys.version)
!pip3 show fastai

3.13.3 (v3.13.3:6280bb54784, Apr  8 2025, 10:47:54) [Clang 15.0.0 (clang-1500.3.9.4)]
Name: fastai
Version: 2.8.2
Summary: fastai simplifies training fast and accurate neural nets using modern best practices
Home-page: https://github.com/fastai/fastai
Author: Jeremy Howard, Sylvain Gugger, and contributors
Author-email: info@fast.ai
License: Apache Software License 2.0
Location: /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages
Requires: cloudpickle, fastcore, fastdownload, fastprogress, fasttransform, matplotlib, packaging, pandas, pillow, pip, plum-dispatch, pyyaml, requests, scikit-learn, scipy, spacy, torch, torchvision
Required-by: 
